In [418]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import plotly.io as pio
import pycountry
from wordcloud import *
from functools import reduce
from PIL import Image
pd.set_option('display.max_rows', 200)
from statsmodels.graphics.mosaicplot import mosaic
from plotly.subplots import make_subplots
pio.templates.default = "simple_white"
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import chart_studio.tools as tls

path = './'

#complete DFs
rhz_artworks = pd.read_pickle(path+'Rhizome_data/rhizome_artworks_extra.pkl')
rhz_artworks = rhz_artworks.replace('N/B', 'NB')
rhz_artists = pd.read_pickle(path+'Rhizome_data/rhizome_artists_extra.pkl')
moma_artists = pd.read_pickle(path+'MOMA_data/pickle/MoMAArtists.pkl')
moma_artworks = pd.read_pickle(path+'MOMA_data/pickle/MoMAartworks.pkl')
moma_artworks_old =  pd.read_pickle(path+'MOMA_data/pickle/old_artworks.pkl')
#moma_artworks_old.loc[:,'DateAcquired'] = moma_artworks_old['DateAcquired'].where((moma_artworks_old['DateAcquired'].str.len() <= 4), moma_artworks_old['DateAcquired'].str[0:4])
#moma_artworks_old['DateAcquired'] = moma_artworks_old['DateAcquired'].replace('nan', str('0'))
#moma_artworks_old['DateAcquired']=moma_artworks_old['DateAcquired'].astype('int')
moma_artworks_new = pd.read_pickle(path+'MOMA_data/pickle/new_artworks.pkl')
#moma_artworks_new.loc[:,'DateAcquired'] = moma_artworks_new['DateAcquired'].where((moma_artworks_new['DateAcquired'].str.len() <= 4), moma_artworks_new['DateAcquired'].str[0:4])
#moma_artworks_new['DateAcquired'] = moma_artworks_new['DateAcquired'].replace('nan', str('0'))
#moma_artworks_new['DateAcquired']=moma_artworks_new['DateAcquired'].astype('int')
#moma_artworks = pd.concat([moma_artworks_old, moma_artworks_new])
#moma_artworks_old = moma_artworks.loc[moma_artworks['DateCreated'] <= 1982]
#moma_artworks_new = moma_artworks.loc[moma_artworks['DateCreated'] >= 1983]
moma_rhz_compare = moma_artworks_new.loc[moma_artworks_new['dateAcquired'] >= 2000]
moma_rhz_compare = moma_rhz_compare.loc[moma_rhz_compare['dateCreated'] >= 1983]

#MoMA department DFs
moma_draw = moma_artworks.loc[moma_artworks['Department'] == 'Drawings & Prints']
moma_draw_before = moma_draw.loc[moma_draw['dateCreated'] <= 1982]
moma_draw_after = moma_draw.loc[moma_draw['dateCreated'] >= 1983]
moma_films = moma_artworks.loc[moma_artworks['Department'] == 'Film']
moma_films_before = moma_films.loc[moma_films['dateCreated'] <= 1982]
moma_films_after = moma_films.loc[moma_films['dateCreated'] >= 1983]
moma_media = moma_artworks.loc[moma_artworks['Department'] == 'Media and Performance']
moma_media_before = moma_media.loc[moma_media['dateCreated'] <= 1982]
moma_media_after = moma_media.loc[moma_media['dateCreated'] >= 1983]
moma_photo = moma_artworks.loc[moma_artworks['Department'] == 'Photography']
moma_photo_before = moma_photo.loc[moma_photo['dateCreated'] <= 1982]
moma_photo_after = moma_photo.loc[moma_photo['dateCreated'] >= 1983]

#Rhizome with text
rhizome_txt_clean = pd.read_pickle(path+'Rhizome_data/rhizome_artworks_extra_text_clean.pkl')
rhizome_txt_stop_kw = pd.read_pickle(path+'Rhizome_data/rhizome_artworks_extra_text_clean_stop_keywords.pkl')

#MoMA with text
moma_arch_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_cont_text_final.pkl')
moma_arch_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_mod_text_only_final.pkl')
moma_draw_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/draws_prints_cont_text_final.pkl')
moma_draw_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/draws_prints_mod_text_final.pkl')
moma_films_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/films_cont_text_final.pkl')
moma_films_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/films_mod_text_final.pkl')
moma_fluxus_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/fluxus_cont_text_final.pkl')
moma_fluxus_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/fluxus_mod_text_final.pkl')
moma_media_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/media_perf_cont_text_final.pkl')
moma_media_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/media_perf_mod_text_final.pkl')
moma_paint_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/paint_sculp_cont_text_final.pkl')
moma_paint_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/paint_sculp_mod_text_final.pkl')
moma_photo_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/photo_cont_text_final.pkl')
moma_photo_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/photo_mod_text_final.pkl')

#moma text stop and keywords
moma_arch_cont_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_cont_text_final_stop.pkl')
moma_arch_mod_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_mod_text_only_final_stop.pkl')
moma_draw_cont_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/draws_prints_cont_text_final_stop.pkl')
moma_draw_mod_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/draws_prints_mod_text_final_stop.pkl')
moma_films_cont_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/films_cont_text_final_stop.pkl')
moma_films_mod_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/films_mod_text_final_stop.pkl')
moma_media_cont_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/media_perf_cont_text_final_stop.pkl')
moma_media_mod_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/media_perf_mod_text_final_stop.pkl')
moma_paint_cont_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/paint_sculp_cont_text_final_stop.pkl')
moma_paint_mod_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/paint_sculp_mod_text_final_stop.pkl')
moma_photo_cont_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/photo_cont_text_final_stop.pkl')
moma_photo_mod_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/photo_mod_text_final_stop.pkl')
moma_full_text_stop = pd.concat([moma_arch_cont_text_stop, moma_draw_cont_text_stop, moma_draw_mod_text_stop, moma_films_cont_text_stop, moma_films_mod_text_stop, moma_media_cont_text_stop, moma_media_mod_text_stop, moma_paint_cont_text_stop, moma_paint_mod_text_stop, moma_photo_cont_text_stop, moma_photo_mod_text_stop])

In [40]:
def group_genders(df):
    collectives = ['M', 'F', 'missing', 'NB']
    df = df['Gender'].value_counts().rename_axis('Gender').reset_index(name='Counts')
    df_collectives = ['collectives', df[~df.Gender.isin(collectives)]['Counts'].sum()]
    df = df.loc[~df.Gender.isin(collectives) == False].reset_index(drop=True)
    df.loc[5] = df_collectives
    df = df.sort_values(by='Counts', ascending=False).replace('M', 'Male').replace('F', 'Female')
    return df

def genders_over_time(df, time_var, time_title):
    df_m = df.loc[df['Gender'] == 'M']
    df_m = df_m[time_var].value_counts().rename_axis(time_title).reset_index(name='Male Artists')
    df_f = df.loc[df['Gender'] == 'F']
    df_f = df_f[time_var].value_counts().rename_axis(time_title).reset_index(name='Female Artists')
    df_merge = pd.merge(df_m, df_f, on=time_title, how='outer')
    df_nb = df.loc[df['Gender'] == nonbinary]
    df_nb = df_nb[time_var].value_counts().rename_axis(time_title).reset_index(name='Non Binary Artists')
    df_col = df[(df['Gender'] != 'M') & (df['Gender'] != 'F') & (df['Gender'] != 'missing') & (df['Gender'] != 'NB')]
    df_col = df_col[time_var].value_counts().rename_axis(time_title).reset_index(name='Collectives')
    df_missing = df.loc[df['Gender'] == 'missing']
    df_missing = df_missing[time_var].value_counts().rename_axis(time_title).reset_index(name='Artists with Missing Gender')
    df_extra = pd.merge(df_nb, df_col, on=time_title, how='outer')
    df_merge = pd.merge(df_merge, df_extra, on=time_title, how='outer')
    df_merge = pd.merge(df_merge, df_missing, on=time_title, how='outer')
    df_merge = df_merge.fillna('0')
    df_merge = df_merge.loc[df_merge[time_title] != 0]
    df_merge = df_merge.astype(int)
    df_merge = df_merge.sort_values(by=time_title).reset_index(drop=True)
    return df_merge

def getContinent(string):
    if string in north:
        return 'North Globe'
    elif string in south:
        return 'South Globe'
    else: 
        'missing'

def do_fuzzy_search(country):
    try:
        result = pycountry.countries.search_fuzzy(country)
        return result[0].alpha_3
    except:
        return np.nan

**GENDER BREAKDOWN PIE CHARTS RHZ VS MoMA FULL + RHZ VS MoMA SAMPLED**

In [224]:
#create a DF w/ genders and counts, group collectives together
rhz_gender_rep = group_genders(rhz_artworks)
moma_gender_rep = group_genders(moma_artworks)

fig_1 = px.pie(rhz_gender_rep, values='Counts', names='Gender',
             title='Gender Breakdown of Rhizome Collection')
fig_1.update_traces(textposition='auto', textinfo='label+percent', hoverinfo='label+value+percent', hovertemplate=None, pull=[0, 0, 0, 0.2])
fig_1.update_layout(width=800)

fig_2 = px.pie(moma_gender_rep, values='Counts', names='Gender',
             title='Gender Breakdown of MoMA Collection')
fig_2.update_traces(textposition='auto', textinfo='label+percent', hoverinfo='label+value+percent', hovertemplate=None, pull=[0, 0, 0, 0.2])
fig_2.update_layout(width=800)

fig_1.show(), fig_2.show()

(None, None)

In [226]:
#create a DF w/ genders and counts, group collectives together
rhz_gender_rep = group_genders(rhz_artworks)
moma_rhz_compare_gender_rep = group_genders(moma_rhz_compare)

fig_1 = px.pie(rhz_gender_rep, values='Counts', names='Gender',
             title='Gender Breakdown of Rhizome Collection')
fig_1.update_traces(textposition='auto', textinfo='label+percent', hoverinfo='label+value+percent', hovertemplate=None, pull=[0, 0, 0, 0.2])
fig_1.update_layout(width=800)

fig_2 = px.pie(moma_rhz_compare_gender_rep, values='Counts', names='Gender',
             title='Gender Breakdown of MoMA Collection (Sampled)')
fig_2.update_traces(textposition='auto', textinfo='label+percent', hoverinfo='label+value+percent', hovertemplate=None, pull=[0, 0, 0, 0.2])
fig_2.update_layout(width=800)

fig_1.show(), fig_2.show()

(None, None)

**SUNBURST CHARTS FOR COLLECTIVE GENDER BREAKDOWN**

In [228]:
rhizome_gender_counts = rhz_artworks['Gender'].value_counts()
#choose only collectives
rhizome_gender_collectives = rhizome_gender_counts[3:].reset_index()
#group collectives categories
rhizome_gender_collectives_male = rhizome_gender_collectives[(rhizome_gender_collectives['index'].str.contains('F') == False) & (rhizome_gender_collectives['index'].str.contains('missing') == False)]
rhizome_gender_collectives_female =rhizome_gender_collectives[(rhizome_gender_collectives['index'].str.contains('M') == False) & (rhizome_gender_collectives['index'].str.contains('missing') == False)]
rhizome_gender_collectives_both = rhizome_gender_collectives[(rhizome_gender_collectives['index'].str.contains('F')== True) & (rhizome_gender_collectives['index'].str.contains('M')== True) & (rhizome_gender_collectives['index'].str.contains('missing')== False)]
rhizome_gender_collectives_male_missing = rhizome_gender_collectives[(rhizome_gender_collectives['index'].str.contains('M') == True) & (rhizome_gender_collectives['index'].str.contains('missing') == True) & (rhizome_gender_collectives['index'].str.contains('F') == False)]
rhizome_gender_collectives_female_missing = rhizome_gender_collectives[(rhizome_gender_collectives['index'].str.contains('M') == False) & (rhizome_gender_collectives['index'].str.contains('missing') == True) & (rhizome_gender_collectives['index'].str.contains('F') == True)]
rhizome_gender_collectives_both_missing = rhizome_gender_collectives[(rhizome_gender_collectives['index'].str.contains('F')== True) | (rhizome_gender_collectives['index'].str.contains('M')== True) | (rhizome_gender_collectives['index'].str.contains('missing')== True)]
#send filters to series 
rhizome_gender_collectives_both_counts = rhizome_gender_collectives_both.sum()
rhizome_gender_collectives_both_missing_counts = rhizome_gender_collectives_both_missing.sum()
rhizome_gender_collectives_male_counts = rhizome_gender_collectives_male.sum()
rhizome_gender_collectives_female_counts = rhizome_gender_collectives_female.sum()
rhizome_gender_collectives_male_missing_counts = rhizome_gender_collectives_male_missing.sum()
rhizome_gender_collectives_female_missing_counts = rhizome_gender_collectives_female_missing.sum()
#create new DF w/ results 
rhizome_gender_collectives_counts_final = pd.DataFrame()
rhizome_gender_collectives_counts_final['Male Only'] = rhizome_gender_collectives_male_counts
rhizome_gender_collectives_counts_final['Male & Missing'] = rhizome_gender_collectives_male_missing_counts
rhizome_gender_collectives_counts_final['Female Only'] = rhizome_gender_collectives_female_counts
rhizome_gender_collectives_counts_final['Female & Missing'] = rhizome_gender_collectives_female_missing_counts
rhizome_gender_collectives_counts_final['Mixed Gender'] = rhizome_gender_collectives_both_counts
rhizome_gender_collectives_counts_final['Mixed Gender incl. Missing'] = rhizome_gender_collectives_female_missing_counts
rhizome_gender_collectives_counts_final = rhizome_gender_collectives_counts_final.drop('index', axis=0)

In [229]:
moma_gender_counts = moma_rhz_compare['Gender'].value_counts()
#choose only collectives
moma_gender_collectives = moma_gender_counts[3:].reset_index()
#group collectives categories
moma_gender_collectives_male = moma_gender_collectives[(moma_gender_collectives['index'].str.contains('F') == False) & (moma_gender_collectives['index'].str.contains('missing') == False) & (moma_gender_collectives['index'].str.contains('NB') == False)]
moma_gender_collectives_female =moma_gender_collectives[(moma_gender_collectives['index'].str.contains('M') == False) & (moma_gender_collectives['index'].str.contains('missing') == False) & (moma_gender_collectives['index'].str.contains('NB') == False)]
moma_gender_collectives_both = moma_gender_collectives[(moma_gender_collectives['index'].str.contains('F')== True) & (moma_gender_collectives['index'].str.contains('M')== True) & (moma_gender_collectives['index'].str.contains('missing')== False)]
moma_gender_collectives_male_missing = moma_gender_collectives[(moma_gender_collectives['index'].str.contains('M') == True) & (moma_gender_collectives['index'].str.contains('missing') == True) & (moma_gender_collectives['index'].str.contains('F') == False)]
moma_gender_collectives_female_missing = moma_gender_collectives[(moma_gender_collectives['index'].str.contains('M') == False) & (moma_gender_collectives['index'].str.contains('missing') == True) & (moma_gender_collectives['index'].str.contains('F') == True)]
moma_gender_collectives_both_missing = rhizome_gender_collectives[(rhizome_gender_collectives['index'].str.contains('F')== True) | (rhizome_gender_collectives['index'].str.contains('M')== True) | (rhizome_gender_collectives['index'].str.contains('missing')== True)]
#send filters to series 
moma_gender_collectives_both_counts = moma_gender_collectives_both.sum()
moma_gender_collectives_male_counts = moma_gender_collectives_male.sum()
moma_gender_collectives_female_counts = moma_gender_collectives_female.sum()
moma_gender_collectives_male_missing_counts = moma_gender_collectives_male_missing.sum()
moma_gender_collectives_female_missing_counts = moma_gender_collectives_female_missing.sum()
moma_gender_collectives_both_missing_counts = moma_gender_collectives_both_missing.sum()
#create new DF w/ results 
moma_gender_collectives_counts_final = pd.DataFrame()
moma_gender_collectives_counts_final['Male Only'] = moma_gender_collectives_male_counts
moma_gender_collectives_counts_final['Male & Missing'] = moma_gender_collectives_male_missing_counts
moma_gender_collectives_counts_final['Female Only'] = moma_gender_collectives_female_counts
moma_gender_collectives_counts_final['Female & Missing'] = moma_gender_collectives_female_missing_counts
moma_gender_collectives_counts_final['Mixed Gender'] = moma_gender_collectives_both_counts
moma_gender_collectives_counts_final['Mixed Gender incl. Missing'] = moma_gender_collectives_both_missing_counts
moma_gender_collectives_counts_final = moma_gender_collectives_counts_final.drop('index', axis=0)

In [230]:
#collective insight RHIZOME
import plotly.graph_objects as go

fig_sun =go.Figure(go.Sunburst(
    labels=["Collectives", "Males", "Male Only", "Male & Missing", "Females", "Female Only", "Female & Missing", "Mixed Gender", 'Including Missing Gender'],
    parents=["", "Collectives","Males", "Males", "Collectives", "Females", "Females", "Collectives", 'Mixed Gender'],
    values=[133, 39, 37, 2, 11, 8, 3, 83, 3]
),layout=dict(title=dict(text="Gender Breakdown of Rhizome Collectives")))
fig_sun.update_traces(textinfo='label+value', hoverinfo='label+value', hovertemplate=None)
fig_sun.update_layout(width=1000, height=1200)

fig_sun_2 =go.Figure(go.Sunburst(
    labels=["Collectives", "Males", "Male Only", "Male & Missing", "Females", "Female Only", "Female & Missing", "Mixed Gender", 'Including Missing Gender'],
    parents=["", "Collectives","Males", "Males", "Collectives", "Females", "Females", "Collectives", 'Mixed Gender'],
    values=[1634, 903, 435, 468, 206, 135, 71, 525, 131]
),layout=dict(title=dict(text="Gender Breakdown of MoMA Collectives")))
fig_sun_2.update_traces(textinfo='label+value', hoverinfo='label+value', hovertemplate=None)
fig_sun_2.update_layout(width=1000, height=1200)

fig_sun.show(), fig_sun_2.show()

(None, None)

**GENDER BREAKDOWN PIE CHARTS MoMA B4 VS MoMA AFTER**

In [233]:
#create a DF w/ genders and counts, group collectives together
moma_b4_gender_rep = group_genders(moma_artworks_old)
moma_after_gender_rep = group_genders(moma_artworks_new)

fig_1 = px.pie(moma_b4_gender_rep, values='Counts', names='Gender',
             title='Gender Breakdown of MoMA Collection Before 1983')
fig_1.update_traces(textposition='auto', textinfo='label+percent', hoverinfo='label+value+percent', hovertemplate=None, pull=[0, 0, 0, 0.2])
fig_1.update_layout(width=800)

fig_2 = px.pie(moma_after_gender_rep, values='Counts', names='Gender',
             title='Gender Breakdown of MoMA Collection After 1983')
fig_2.update_traces(textposition='auto', textinfo='label+percent', hoverinfo='label+value+percent', hovertemplate=None, pull=[0, 0, 0, 0.2])
fig_2.update_layout(width=800)

fig_1.show(), fig_2.show()

(None, None)

**GENDER REP MoMA BY DEPT BAR CHARTS**

In [3]:
moma_draw_before_gender_rep = group_genders(moma_draw_before)
moma_draw_after_gender_rep = group_genders(moma_draw_after)
moma_films_before_gender_rep = group_genders(moma_films_before)
moma_films_after_gender_rep = group_genders(moma_films_after)
moma_media_before_gender_rep = group_genders(moma_media_before)
moma_media_after_gender_rep = group_genders(moma_media_after)
moma_photo_before_gender_rep = group_genders(moma_photo_before)
moma_photo_after_gender_rep = group_genders(moma_photo_after)

In [4]:
moma_draw_before_gender_rep['Period'] = 'Before 1983'
moma_draw_after_gender_rep['Period'] = 'After 1983'
moma_draw_before_gender_rep['Department'] = 'Drawing & Print'
moma_draw_after_gender_rep['Department'] = 'Drawing & Print'
moma_draw_before_gender_rep['Counts'] = moma_draw_before_gender_rep['Counts'].div(100).round(2)
moma_draw_before_gender_rep['Counts'] = moma_draw_before_gender_rep['Counts'].multiply(30).round(2)
moma_draw_before_gender_rep['Counts'] = moma_draw_before_gender_rep['Counts'].astype(int)
moma_draw_gender_rep_all = pd.concat([moma_draw_before_gender_rep, moma_draw_after_gender_rep])
moma_films_before_gender_rep['Period'] = 'Before 1983'
moma_films_after_gender_rep['Period'] = 'After 1983'
moma_films_before_gender_rep['Department'] = 'Films'
moma_films_after_gender_rep['Department'] = 'Films' 
moma_films_before_gender_rep['Counts'] = moma_films_before_gender_rep['Counts'].div(100).round(2)
moma_films_before_gender_rep['Counts'] = moma_films_before_gender_rep['Counts'].multiply(30).round(2)
moma_films_before_gender_rep['Counts'] = moma_films_before_gender_rep['Counts'].astype(int)
moma_films_gender_rep_all = pd.concat([moma_films_before_gender_rep, moma_films_after_gender_rep])
moma_media_before_gender_rep['Period'] = 'Before 1983'
moma_media_after_gender_rep['Period'] = 'After 1983'
moma_media_before_gender_rep['Department'] = 'Media & Performance'
moma_media_after_gender_rep['Department'] = 'Media & Performance' 
moma_media_before_gender_rep['Counts'] = moma_media_before_gender_rep['Counts'].div(100).round(2)
moma_media_before_gender_rep['Counts'] = moma_media_before_gender_rep['Counts'].multiply(30).round(2)
moma_media_before_gender_rep['Counts'] = moma_media_before_gender_rep['Counts'].astype(int)
moma_media_gender_rep_all = pd.concat([moma_media_before_gender_rep, moma_media_after_gender_rep])
moma_photo_before_gender_rep['Period'] = 'Before 1983'
moma_photo_after_gender_rep['Period'] = 'After 1983'
moma_photo_before_gender_rep['Department'] = 'Photography'
moma_photo_after_gender_rep['Department'] = 'Photography' 
moma_photo_before_gender_rep['Counts'] = moma_photo_before_gender_rep['Counts'].div(100).round(2)
moma_photo_before_gender_rep['Counts'] = moma_photo_before_gender_rep['Counts'].multiply(30).round(2)
moma_photo_before_gender_rep['Counts'] = moma_photo_before_gender_rep['Counts'].astype(int)
moma_photo_gender_rep_all = pd.concat([moma_photo_before_gender_rep, moma_photo_after_gender_rep])
moma_depts_gender_rep_all = pd.concat([moma_photo_gender_rep_all, moma_media_gender_rep_all, moma_films_gender_rep_all, moma_draw_gender_rep_all])
moma_depts_gender_rep_all = moma_depts_gender_rep_all.reset_index(drop='True')

In [5]:
fig = px.bar(moma_depts_gender_rep_all, x="Gender", y="Counts", color="Period", facet_col="Department", facet_col_spacing=0.06,  facet_col_wrap=2, title="Gender Rep in MoMA Departments before and after 1983", barmode='stack', log_y=True, text_auto=True, category_orders={"Period": ["Before 1983", 'After 1983']})
fig.update_xaxes(matches=None, showticklabels=True, categoryorder='array', categoryarray=['Male', 'Female', 'collectives', 'missing', 'NB'])
fig.update_yaxes(showticklabels=True)
fig.update_traces(textposition="outside")
fig.update_layout(height=800, width=1600)
fig.show()

**GENDER REP RHZ VS MOMA SAMPLE BY DEPT**

In [7]:
#create a list of rhizome mediums based on keywords research 
rhz_medium = rhizome_txt_stop_kw.copy()
rhz_medium = rhz_medium.replace('N/B', 'NB')
# create a list of our conditionsP
conditions = [
    (rhz_medium['Text'].str.contains('video|film|animation|movie|tv', regex=True)),
    (rhz_medium['Text'].str.contains('images|image|photographs|photography|photo|photos|painting|drawing', regex=True)),
    (rhz_medium['Text'].str.contains('installation|performance|sound|audio|song|music', regex=True)),
    (rhz_medium['Text'].str.contains('website|flash|java|data|database|webpage|net|internet|site|sites|online|webbased|augmented|virtual|reality|game|interaction|interactive|interface|project|xml|mashup|map|generative|wifi|network|email|computer', regex=True)),
    ]

# create a list of the values we want to assign for each condition
values = ['Moving Images', 'Still Images', 'Installation & Performance', 'Net Art']

# create a new column and use np.select to assign values to it using our lists as arguments
rhz_medium['Medium'] = np.select(conditions, values)

# display updated DataFrame
rhz_medium['Medium'] = rhz_medium['Medium'].replace('0', 'missing')
rhz_medium['Medium'] = rhz_medium['Medium'].astype(str)


In [8]:
rhz_gender_rep_mi = group_genders(rhz_medium.loc[rhz_medium['Medium'] == 'Moving Images'])
rhz_gender_rep_img = group_genders(rhz_medium.loc[rhz_medium['Medium'] == 'Still Images'])
rhz_gender_rep_intall = group_genders(rhz_medium.loc[rhz_medium['Medium'] == 'Installation & Performance'])
rhz_gender_rep_netart = group_genders(rhz_medium.loc[rhz_medium['Medium'] == 'Net Art'])
rhz_gender_rep_mi['Department'] = 'Moving Images'
rhz_gender_rep_img['Department'] = 'Still Images'
rhz_gender_rep_intall['Department'] = 'Installation'
rhz_gender_rep_netart['Department'] = 'Net Art'
rhz_rep_all = pd.concat([rhz_gender_rep_mi, rhz_gender_rep_img, rhz_gender_rep_intall, rhz_gender_rep_netart])
rhz_rep_all['Source'] = 'Rhizome'

In [11]:
moma_depts_gender_rep_all = moma_depts_gender_rep_all.replace(['Photography', 'Drawing & Print'], 'Still Images').groupby(['Department', 'Period', 'Gender'],as_index=False).agg({'Counts': 'sum'})

In [ ]:
moma_depts_gender_rep_all_2 = moma_depts_gender_rep_all.loc[moma_depts_gender_rep_all['Period'] == 'After 1983']
moma_depts_gender_rep_all_2['Counts'] = moma_depts_gender_rep_all_2['Counts'].div(10).round(2)
moma_depts_gender_rep_all_2['Counts'] = moma_depts_gender_rep_all_2['Counts'].astype(int)
moma_depts_gender_rep_all_2['Source'] = 'MoMA'
moma_rhz_gender_compare = pd.concat([rhz_rep_all, moma_depts_gender_rep_all_2])
moma_rhz_gender_compare = moma_rhz_gender_compare.drop('Period', axis=1)
moma_rhz_gender_compare = moma_rhz_gender_compare.replace(['Films'], 'Moving Images').groupby(['Department', 'Source', 'Gender'],as_index=False).agg({'Counts': 'sum'})
moma_rhz_gender_compare = moma_rhz_gender_compare.replace(['Net Art', 'Installation'], 'Media & Performance').groupby(['Department', 'Source', 'Gender'],as_index=False).agg({'Counts': 'sum'})

In [280]:
fig = px.bar(moma_rhz_gender_compare, x="Gender", y="Counts", color="Department", facet_col="Source", facet_col_spacing=0.06,  facet_col_wrap=2, title="Gender Rep Between Rhizome and MoMA Departments", barmode='group', log_y=True, text_auto=True, category_orders={"Department": ["Still Images", "Moving Images", 'Media & Performance']})
fig.update_xaxes(matches=None, showticklabels=True,  categoryorder='array', categoryarray=['Male', 'Female', 'collectives', 'missing', 'NB'])
fig.update_yaxes(showticklabels=True)
fig.update_traces(textposition="outside")
fig.update_layout(height=800, width=1600)
fig.show()

**NATIONALITIES**
- NORMALIZE NATIONALITY SPELLING AND COLLECTIVE SPELLING ACROSS BOTH
- FINDING NAN ISSUE WITH COUNTS

In [ ]:
# create working df for MoMA sam create dictionary keeping saved the 'others' nationalities
nationalities = ', '.join(moma_rhz_compare.Nationality)
nationalities = list(set(nationalities.split(', ')))
df_moma_complete = pd.DataFrame(columns= ["Nationality", "dateAcquired", "Count", "Females", "Males", 'Missing', 'Non-Binary']) 

OthersMoma = dict()
for nat in nationalities:
    nat_sub =  moma_rhz_compare[moma_rhz_compare.Nationality == nat]   
     
    nat_sub.dateAcquired = nat_sub.dateAcquired.astype('str')
    years = ', '.join(nat_sub.dateAcquired)
    years = sorted(list([item[:4] for item in list(set(years.split(', ')))]))[0:-1]
    for year in years:
        year_sub = nat_sub[nat_sub.dateAcquired == year]
        entries_year = len(year_sub)
        f_count = len(year_sub[year_sub.Gender == 'F'])
        m_count = len(year_sub[year_sub.Gender == 'M'])
        missing_count = len(year_sub[year_sub.Gender == 'missing'])
        nonbinary_count = len(year_sub[year_sub.Gender == 'NB'])
        df_moma_complete.loc[len(df_moma_complete.index)] = [nat, year, entries_year, f_count, m_count, missing_count, nonbinary_count]
  

df_moma_complete.dateAcquired = df_moma_complete.dateAcquired.astype('int')
# replot dataset any nationality that has count < 5 becomes 'Others'
#MC_working = df_moma_complete[df_moma_complete.dateAcquired > 1929]
#MC_working.dateAcquired.astype('int')
MC_working_final = df_moma_complete.groupby(['Nationality','dateAcquired'],as_index=False).agg({'Count': 'sum','Females': 'sum','Males': 'sum','Missing': 'sum','Non-Binary': 'sum'})

# create MOMA df with country names instead of nationality
natios_MOMA = set(df_moma_complete.Nationality)

missing = pd.DataFrame({'Aalborgenser': ['Korean', 'Native American', 'Canadian Inuit'], 'Aalborg': ['Korea', 'United States', 'Canada']})
missing
df_natParse = pd.read_csv('https://raw.githubusercontent.com/knowitall/chunkedextractor/master/src/main/resources/edu/knowitall/chunkedextractor/demonyms.csv')
correct_country = pd.concat([missing, df_natParse])
Country_df=df_moma_complete.copy()

for item in natios_MOMA:
    my = correct_country[correct_country['Aalborgenser'] == item]
    
    country = my[:1]['Aalborg'].values
    
    if len(country)>0:
        country_str = my[:1]['Aalborg'].values[0]

        Country_df.loc[Country_df["Nationality"] == item, "Nationality"] = country_str

MC_countries_count = pd.DataFrame(columns= ['Nation', 'Count'])

new_set = set(Country_df.Nationality)
for item in new_set:
    subCountry = Country_df[Country_df['Nationality'] == item]
    sum_acquisitions = subCountry['Count'].sum()
    MC_countries_count.loc[len(MC_countries_count.index)] = [item, sum_acquisitions]

MC_countries_count["country_code"] = MC_countries_count["Nation"].apply(lambda country: do_fuzzy_search(country))
MC_countries_count['Count'] = MC_countries_count['Count'].div(10).round(2)
MC_countries_count['Count'] = MC_countries_count['Count'].astype(int)

In [237]:
# create moma map plot

fig = go.Figure(data=go.Choropleth(
    locations = MC_countries_count['country_code'],
    z = MC_countries_count['Count'],
    text = MC_countries_count['Nation'],
    colorscale=[
            [0,"#8e5a79"],
            [0.3 ,"#925f7d"],
            [0.325 ,"#966582"],
            [0.350 ,"#9a6a87"],
            [0.375,"#9e708b"],
            [0.4 ,"#a27590"],
            [0.525 ,"#b690a6"],
            [0.550 ,"#ba96ab"],
            [0.575,"#be9baf"],
            [0.6 ,"#c2a0b4"],
            [0.625 ,"#c6a6b8"],
            [0.650 ,"#caabbd"],
            [0.675,"#ceb1c2"],
            [0.7 ,"#d2b6c6"],
            [0.725 ,"#d6bccb"],
            [0.995 ,"#d9c1cf"],
            [1, "#FFFFFF"]],
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_title = 'Total artworks',
))

fig.update_layout(
    title_text='Spread of MoMA Collection (Sampled)',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='natural earth',
        showocean=True, oceancolor="LightBlue"
    ),
    height = 700,
)


fig.show()

In [ ]:
# create working df for RHZ create dictionary keeping saved the 'others' nationalities
nationalities = ', '.join(rhz_artworks.Nationality)
nationalities = list(set(nationalities.split(', ')))
rhz_complete = pd.DataFrame(columns= ["Nationality", "dateAcquired", "Count", "Females", "Males", 'Missing', 'Non-Binary']) 

OthersRHZ = dict()
for nat in nationalities:
    nat_sub =  rhz_artworks[rhz_artworks.Nationality == nat]   
     
    nat_sub.dateAcquired = nat_sub.dateAcquired.astype('str')
    years = ', '.join(nat_sub.dateAcquired)
    years = sorted(list([item[:4] for item in list(set(years.split(', ')))]))[0:-1]
    for year in years:
        year_sub = nat_sub[nat_sub.dateAcquired == year]
        entries_year = len(year_sub)
        f_count = len(year_sub[year_sub.Gender == 'F'])
        m_count = len(year_sub[year_sub.Gender == 'M'])
        missing_count = len(year_sub[year_sub.Gender == 'missing'])
        nonbinary_count = len(year_sub[year_sub.Gender == 'NB'])
        rhz_complete.loc[len(rhz_complete.index)] = [nat, year, entries_year, f_count, m_count, missing_count, nonbinary_count]
  

rhz_complete.dateAcquired = rhz_complete.dateAcquired.astype('int')
RHZ_working_final = rhz_complete.groupby(['Nationality','dateAcquired'],as_index=False).agg({'Count': 'sum','Females': 'sum','Males': 'sum','Missing': 'sum','Non-Binary': 'sum'})


# create MOMA df with country names instead of nationality
natios_RHZ = set(rhz_complete.Nationality)

missing = pd.DataFrame({'Aalborgenser': ['Korean', 'Native American', 'Canadian Inuit'], 'Aalborg': ['Korea', 'United States', 'Canada']})
missing
df_natParse = pd.read_csv('https://raw.githubusercontent.com/knowitall/chunkedextractor/master/src/main/resources/edu/knowitall/chunkedextractor/demonyms.csv')
correct_country = pd.concat([missing, df_natParse])
Country_df=rhz_complete.copy()

for item in natios_RHZ:
    my = correct_country[correct_country['Aalborgenser'] == item]
    
    country = my[:1]['Aalborg'].values
    
    if len(country)>0:
        country_str = my[:1]['Aalborg'].values[0]

        Country_df.loc[Country_df["Nationality"] == item, "Nationality"] = country_str

RHZ_countries_count = pd.DataFrame(columns= ['Nation', 'Count'])

new_set = set(Country_df.Nationality)
for item in new_set:
    subCountry = Country_df[Country_df['Nationality'] == item]
    sum_acquisitions = subCountry['Count'].sum()
    RHZ_countries_count.loc[len(RHZ_countries_count.index)] = [item, sum_acquisitions]

RHZ_countries_count["country_code"] = RHZ_countries_count["Nation"].apply(lambda country: do_fuzzy_search(country))

In [238]:
# create rhizome map plot

fig = go.Figure(data=go.Choropleth(
    locations = RHZ_countries_count['country_code'],
    z = RHZ_countries_count['Count'],
    text = RHZ_countries_count['Nation'],
    colorscale=[
            [0,"#8e5a79"],
            [0.3 ,"#925f7d"],
            [0.325 ,"#966582"],
            [0.350 ,"#9a6a87"],
            [0.375,"#9e708b"],
            [0.4 ,"#a27590"],
            [0.525 ,"#b690a6"],
            [0.550 ,"#ba96ab"],
            [0.575,"#be9baf"],
            [0.6 ,"#c2a0b4"],
            [0.625 ,"#c6a6b8"],
            [0.650 ,"#caabbd"],
            [0.675,"#ceb1c2"],
            [0.7 ,"#d2b6c6"],
            [0.725 ,"#d6bccb"],
            [0.995 ,"#d9c1cf"],
            [1, "#FFFFFF"]],
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_title = 'Total artworks',
))

fig.update_layout(
    title_text='Spread of Rhizome Collection',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='natural earth',
        showocean=True, oceancolor="LightBlue"
    ),
    height = 700,
)


fig.show()

In [38]:
#continents of nats between moma compare e rhizome found typos

north= ['Australian','New Zealander','Guatemalan','Canadian', 'Cuban','American', 'American/Canadian','Israeli','Japanese', 'Korean', 'South Korean','Albanian', 'Austrian', 'Belgian', 'British', 'Bulgarian', 'Croatian','Danish', 'Dutch', 'Finnish','French','German','Greek', 'Hungarian','Irish', 'Italian', 'Norwegian','Polish', 'Portuguese', 'Romanian', 'Russian','Scottish','Slovenian', 'Spanish','Swedish','Swiss','Ukrainian','Mexican', 'Turkish']
south = ['Argentine', 'Brazilian','Chilean', 'Colombian', 'Salvadoran', 'Uruguayan','Venezuelan''Native American','Bahamian','Algerian', 'Bangladeshi', 'Emirati','Indian', 'Iranian','Chinese','Beninese', 'Cameroonian',  'Congolese','Egyptian','Ethiopian','Filipino''Ivorian', 'Kenyan', 'Moroccan','South African','Sudanese','Zimbabwean','Bosnian']
#Mixed  = ['Afghan/American', 'Dutch/Brazilian', 'Iranian/Canadian''British/Canadian']
Nope = ['Nationality unknown', 'missing']

In [41]:
#make working df for moma complete
moma_artworks_old['Period'] ='Before 1983'
moma_artworks_new['Period'] = 'After 1983'
Moma_by_periods = pd.concat([moma_artworks_old,moma_artworks_new])
department = ['Film','Media and Performance', 'Drawings & Prints', 'Photography']
df_Moma_by_periods = pd.DataFrame(columns= ['Continent', 'Nationality', 'Department','Period', "Count", "Females", "Males", 'Missing', 'Non-Binary', 'Collectives']) 

OthersMoma = dict()
for dep in department:
    dep_sub =  Moma_by_periods[Moma_by_periods.Department == dep]  
    nats = set(', '.join(dep_sub.Nationality).split(', '))
    for nat in nats:
        continent = getContinent(nat)
        nats_sub = dep_sub[dep_sub.Nationality.str.contains(nat, regex= True, na=False)]
        periods = ', '.join(nats_sub.Period)
        periods = set(', '.join(nats_sub.Period).split(', '))
        for period in periods:
            final_sub = nats_sub[nats_sub['Period']==period]
            tot = len(final_sub) 
            period = final_sub.Period.values[0]
            f_count = len(final_sub[final_sub.Gender == 'F'])
            m_count = len(final_sub[final_sub.Gender == 'M'])
            missing_count = len(final_sub[final_sub.Gender == 'missing'])
            nonbinary_count = len(final_sub[final_sub.Gender == 'NB'])
            collectives_count = len(final_sub[final_sub.Gender.str.contains(',', regex= True, na=False)])
            df_Moma_by_periods.loc[len(df_Moma_by_periods.index)] = [continent, nat, dep,period, tot, f_count, m_count, missing_count, nonbinary_count, collectives_count]

df_Moma_by_periods = df_Moma_by_periods.groupby(['Continent', 'Nationality','Department', 'Period'],as_index=False).agg({'Count': 'sum','Females': 'sum','Males': 'sum','Missing': 'sum','Non-Binary': 'sum', 'Collectives': 'sum'})


In [42]:
#make working df for rhz
rhz_nationalities = rhz_medium.copy()
rhz_nationalities['Source'] = 'Rhizome'
rhz_nationalities.rename(columns = {'Medium':'Department'}, inplace = True)
rhz_nationalities = rhz_nationalities.replace('N/B', 'NB')
moma_compare = moma_rhz_compare.copy()
moma_compare['Source'] = 'MoMA'
moma_rhz_nats = pd.concat([moma_compare,rhz_nationalities])
department_2 = ['Film','Media and Performance', 'Drawings & Prints', 'Photography', 'Moving Images', 'Still Images', 'Installation & Performance', 'Net Art']
moma_rhz_nats_compare = pd.DataFrame(columns= ['Continent', 'Nationality', 'Department', 'Source', 'Gender' "Count", "Females", "Males", 'Missing', 'Non-Binary', 'Collectives']) 

OthersMomaRhz = dict()
for dep in department_2:
    dep_sub =  moma_rhz_nats[moma_rhz_nats.Department == dep]  
    nats = set(', '.join(dep_sub.Nationality).split(', '))
    for nat in nats:
        continent = getContinent(nat)
        nats_sub = dep_sub[dep_sub.Nationality.str.contains(nat, regex= True, na=False)]
        sources = ', '.join(nats_sub.Source)
        sources = set(', '.join(nats_sub.Source).split(', '))
        for source in sources:
            final_sub = nats_sub[nats_sub['Source']==source]
            tot = len(final_sub) 
            f_count = len(final_sub[final_sub.Gender == 'F'])
            m_count = len(final_sub[final_sub.Gender == 'M'])
            missing_count = len(final_sub[final_sub.Gender == 'missing'])
            nonbinary_count = len(final_sub[final_sub.Gender == 'NB'])
            collectives_count = len(final_sub[final_sub.Gender.str.contains(',', regex= True, na=False)])
            moma_rhz_nats_compare.loc[len(moma_rhz_nats_compare.index)] = [continent, nat, dep, source, tot, f_count, m_count, missing_count, nonbinary_count, collectives_count]

moma_rhz_nats_compare = moma_rhz_nats_compare.groupby(['Continent', 'Nationality','Department', 'Source', 'Gender'],as_index=False).agg({'Count': 'sum','Females': 'sum','Males': 'sum','Missing': 'sum','Non-Binary': 'sum', 'Collectives': 'sum'})


In [ ]:
#make the two periods comparable
old = df_Moma_by_periods[df_Moma_by_periods['Period'] == 'Before 1983']
old['Count'] = old['Count'].div(100).round(2)
old['Count'] = old['Count'].multiply(30).round(2)
old['Count'] = old['Count'].astype('int')
new = df_Moma_by_periods[df_Moma_by_periods['Period'] == 'After 1983']

df_Moma_by_periods_compare = pd.concat([old,new])

In [ ]:
#make the moma and rhizome comparable
moma_normalise = moma_rhz_nats_compare.loc[moma_rhz_nats_compare['Source'] == 'MoMA']
moma_normalise['Count'] = moma_normalise['Count'].div(10).round(2)
moma_normalise['Count'] = moma_normalise['Count'].astype('int')
rhz_no_moma = moma_rhz_nats_compare.loc[moma_rhz_nats_compare['Source'] == 'Rhizome']

Compare_nats_department = pd.concat([moma_normalise, rhz_no_moma])
Compare_nats_department = Compare_nats_department.loc[Compare_nats_department['Continent'] != 'Mixed']
Compare_nats_department = Compare_nats_department.replace(['Net Art', 'Installation & Performance'], 'Media and Performance').groupby(['Continent', 'Nationality', 'Department', 'Source'],as_index=False).agg({'Count': 'sum','Females': 'sum','Males': 'sum','Missing': 'sum','Non-Binary': 'sum', 'Collectives': 'sum'})
Compare_nats_department = Compare_nats_department.replace(['Film'], 'Moving Images').groupby(['Continent', 'Nationality', 'Department', 'Source'],as_index=False).agg({'Count': 'sum','Females': 'sum','Males': 'sum','Missing': 'sum','Non-Binary': 'sum', 'Collectives': 'sum'})
Compare_nats_department = Compare_nats_department.replace(['Photography', 'Drawings & Prints'], 'Still Images').groupby(['Continent', 'Nationality', 'Department', 'Source'],as_index=False).agg({'Count': 'sum','Females': 'sum','Males': 'sum','Missing': 'sum','Non-Binary': 'sum', 'Collectives': 'sum'})

In [ ]:
north_vs_south = Compare_nats_department.groupby(['Continent', 'Source'],as_index=False).agg({'Count': 'sum','Females': 'sum','Males': 'sum','Missing': 'sum','Non-Binary': 'sum'})
fig = px.bar(north_vs_south, x="Continent", y="Count", color='Source', title="TBC", barmode='group', text_auto=True, category_orders={"Department": ["Still Images", 'Photography', 'Drawing & Print',  "Moving Images", 'Films', "Installation", "Net Art", 'Media & Performance']})
fig.update_xaxes(matches=None, showticklabels=True)
fig.update_yaxes(showticklabels=True)
fig.update_traces(textposition="inside")
fig.update_layout(height=800, width=800)
fig.show()

In [294]:
rhz_moma_south = Compare_nats_department.loc[Compare_nats_department['Continent'] == 'South Globe']
rhz_moma_south = rhz_moma_south.loc[rhz_moma_south['Count'] != 0]
rhz_moma_south = rhz_moma_south.groupby(['Continent','Nationality','Source'],as_index=False).agg({'Count': 'sum','Females': 'sum','Males': 'sum','Missing': 'sum','Non-Binary': 'sum'})
fig11 = px.bar(rhz_moma_south, x="Nationality", y="Count", color='Source', facet_col_spacing=0.06,  facet_col_wrap=2, title="Global South Nationalities - Rhizome vs MoMA (sampled)", text='Count', barmode='stack')
fig11.update_xaxes(matches=None, showticklabels=True)
fig11.update_yaxes(showticklabels=True)
fig11.update_traces(textposition='outside', showlegend=False)
fig11.update_layout(height=600)
fig11.show()

In [ ]:
rhz_parallel = rhz_medium.copy()
rhz_parallel = rhz_parallel.drop(['URL', 'Title', 'Artist', 'Text', 'Keywords'], axis=1)
rhz_parallel['Continent'] = rhz_parallel['Nationality'].apply(lambda x:getContinent(x))
rhz_parallel['Continent'] = rhz_parallel['Continent'].astype(str)
rhz_parallel = rhz_parallel.loc[rhz_parallel['Continent'] != 'None']
rhz_parallel = rhz_parallel.replace('North Globe', 0)
rhz_parallel = rhz_parallel.replace('South Globe', 1)
rhz_parallel = rhz_parallel.loc[rhz_parallel['Medium'] != 'missing']
rhz_parallel = rhz_parallel.replace(['Installation & Performance', 'Net Art'], 'Media and Performance')
moma_parallel = moma_rhz_compare.copy()
moma_parallel = moma_parallel.drop(['URL', 'Title', 'Artist', 'Medium', 'ConstituentID', 'ThumbnailURL'], axis=1)
moma_parallel['Continent'] = moma_parallel['Nationality'].apply(lambda x:getContinent(x))
moma_parallel['Continent'] = moma_parallel['Continent'].astype(str)
moma_parallel = moma_parallel.loc[moma_parallel['Continent'] != 'None']
moma_parallel = moma_parallel.replace('North Globe', 0)
moma_parallel = moma_parallel.replace('South Globe', 1)
moma_parallel = moma_parallel.loc[(moma_parallel['Department'] != 'Architecture & Design') & (moma_parallel['Department'] != 'Painting & Sculpture') & (moma_parallel['Department'] != 'Fluxus Collection')]
moma_parallel = moma_parallel.replace('Film', 'Moving Images')
moma_parallel = moma_parallel.replace(['Photography', 'Drawings & Prints'], 'Still Images')

rhz_parallel_both = rhz_parallel.copy()
rhz_parallel_both = rhz_parallel_both.rename(columns={"Medium": "Department"})
rhz_parallel_both['Source'] = 'Rhizome'
moma_parallel_both = moma_parallel.copy()
moma_parallel_both['Source'] = 'MoMA'
moma_rhz_parallel_both = pd.concat([rhz_parallel_both, moma_parallel_both])

moma_rhz_parallel_both_south =  moma_rhz_parallel_both.loc[(moma_rhz_parallel_both['Nationality'] == 'Argentine') | (moma_rhz_parallel_both['Nationality'] == 'Brazilian') | (moma_rhz_parallel_both['Nationality'] == 'Cameroonian') | (moma_rhz_parallel_both['Nationality'] == 'Chilean') | (moma_rhz_parallel_both['Nationality'] == 'Chinese') | (moma_rhz_parallel_both['Nationality'] == 'Colombian') | (moma_rhz_parallel_both['Nationality'] == 'Egyptian') | (moma_rhz_parallel_both['Nationality'] == 'Indian') | (moma_rhz_parallel_both['Nationality'] == 'Iranian') | (moma_rhz_parallel_both['Nationality'] == 'Kenyan') | (moma_rhz_parallel_both['Nationality'] == 'Moroccan') | (moma_rhz_parallel_both['Nationality'] == 'South African') | (moma_rhz_parallel_both['Nationality'] == 'Sudanese') | (moma_rhz_parallel_both['Nationality'] == 'Uruguayan')]
moma_rhz_parallel_both_south

In [ ]:
moma_rhz_parallel_both_south_2 = moma_rhz_parallel_both_south.replace('M', 1)
moma_rhz_parallel_both_south_2 = moma_rhz_parallel_both_south_2.replace('F', 0)
moma_rhz_parallel_both_south_2 = moma_rhz_parallel_both_south_2[(moma_rhz_parallel_both_south_2['Gender'] != 'missing') & (moma_rhz_parallel_both_south_2['Source'] == 'Rhizome')]
moma_rhz_parallel_both_south_2['Gender'] = moma_rhz_parallel_both_south_2['Gender'].astype(int)


# Create dimensions
nat_dim = go.parcats.Dimension(
    values=moma_rhz_parallel_both_south_2.Nationality,
    categoryorder='category ascending', label="Nationality"
)

dept_dim = go.parcats.Dimension(values=moma_rhz_parallel_both_south_2.Department, label="Department")

gender_dim = go.parcats.Dimension(
    values=moma_rhz_parallel_both_south_2.Gender, label="Gender", categoryarray=[0, 1],
    ticktext=['Female', 'Male']
)

# Create parcats trace
color = moma_rhz_parallel_both_south_2.Gender;
colorscale = [[0, 'lightsteelblue'], [1, 'mediumseagreen']];

fig = go.Figure(data = [go.Parcats(dimensions=[gender_dim, dept_dim, nat_dim ],
        line={'color': color, 'colorscale': colorscale},
        hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])
fig.update_layout(height=800, width=1200, title="Gender and Global South Nationalities by Department in Rhizome")
fig.show()

In [ ]:
moma_rhz_parallel_both_south_3 = moma_rhz_parallel_both_south.replace('M', 1)
moma_rhz_parallel_both_south_3 = moma_rhz_parallel_both_south_3.replace('F', 0)
moma_rhz_parallel_both_south_3 = moma_rhz_parallel_both_south_3[(moma_rhz_parallel_both_south_3['Gender'] != 'missing') & (moma_rhz_parallel_both_south_3['Source'] == 'MoMA')]
moma_rhz_parallel_both_south_3['Gender'] = moma_rhz_parallel_both_south_3['Gender'].astype(int)


# Create dimensions
nat_dim = go.parcats.Dimension(
    values=moma_rhz_parallel_both_south_3.Nationality,
    categoryorder='category ascending', label="Nationality"
)

dept_dim = go.parcats.Dimension(values=moma_rhz_parallel_both_south_3.Department, label="Department")

gender_dim = go.parcats.Dimension(
    values=moma_rhz_parallel_both_south_3.Gender, label="Gender", categoryarray=[0, 1],
    ticktext=['Female', 'Male']
)

# Create parcats trace
color = moma_rhz_parallel_both_south_3.Gender;
colorscale = [[0, 'lightsteelblue'], [1, 'mediumseagreen']];

fig = go.Figure(data = [go.Parcats(dimensions=[gender_dim, dept_dim, nat_dim ],
        line={'color': color, 'colorscale': colorscale},
        hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])
fig.update_layout(height=800, width=1200, title="Gender and Global South Nationalities by Department in MoMA (sampled)")
fig.show()

In [ ]:
# create subset for females 
rh = rhz_artworks.copy()

nationalities = ', '.join(rh.Nationality)
nationalities = list(set(nationalities.split(', ')))
df_R_nats_F = pd.DataFrame(columns= ["Nationality", "dateAcquired", "Count", "Gender"]) 

for nat in nationalities:
    nat_sub =  rh[rh.Nationality == nat]
    sub_f = nat_sub[nat_sub.Gender == 'F']   
    sub_f.dateAcquired = sub_f.dateAcquired.astype('str')
    years = ', '.join(sub_f.dateAcquired)
    years = sorted(list([item[:4] for item in list(set(years.split(', ')))]))[0:-1]
    for year in years:
        year_sub = sub_f[sub_f.dateAcquired == year]
        f_count = len(year_sub[year_sub.Gender == 'F'])
        df_R_nats_F.loc[len(df_R_nats_F.index)] = [nat, year, f_count, 'F']

df_R_nats_F.dateAcquired = df_R_nats_F.dateAcquired.astype('int')
df_R_nats_F = df_R_nats_F[df_R_nats_F.dateAcquired >0]

#create subset for males 
df_R_nats_M = pd.DataFrame(columns= ["Nationality", "dateAcquired", "Count", "Gender"]) 

for nat in nationalities:
    nat_sub =  rh[rh.Nationality == nat]
    sub_m = nat_sub[nat_sub.Gender == 'M']   
    sub_m.dateAcquired = sub_m.dateAcquired.astype('str')
    years = ', '.join(sub_m.dateAcquired)
    years = sorted(list([item[:4] for item in list(set(years.split(', ')))]))[0:-1]
    for year in years:
        year_sub = sub_m[sub_m.dateAcquired == year]
        m_count = len(year_sub[year_sub.Gender == 'M'])
        df_R_nats_M.loc[len(df_R_nats_M.index)] = [nat, year, m_count, 'M']

df_R_nats_M.dateAcquired = df_R_nats_M.dateAcquired.astype('int')
df_R_nats_M = df_R_nats_M[df_R_nats_M.dateAcquired >0]

df_R_nats_NB = pd.DataFrame(columns= ["Nationality", "dateAcquired", "Count", "Gender"]) 

for nat in nationalities:
    nat_sub =  rh[rh.Nationality == nat]
    sub_m = nat_sub[nat_sub.Gender == 'NB']   
    sub_m.dateAcquired = sub_m.dateAcquired.astype('str')
    years = ', '.join(sub_m.dateAcquired)
    years = sorted(list([item[:4] for item in list(set(years.split(', ')))]))[0:-1]
    for year in years:
        year_sub = sub_m[sub_m.dateAcquired == year]
        m_count = len(year_sub[year_sub.Gender == 'NB'])
        df_R_nats_NB.loc[len(df_R_nats_NB.index)] = [nat, year, m_count, 'NB']

df_R_nats_NB.dateAcquired = df_R_nats_NB.dateAcquired.astype('int')
df_R_nats_NB = df_R_nats_NB[df_R_nats_NB.dateAcquired >0]

df_R_nats_CL = pd.DataFrame(columns= ["Nationality", "dateAcquired", "Count", "Gender"]) 

for nat in nationalities:
    nat_sub =  rh[rh.Nationality == nat]
    sub_m = nat_sub[nat_sub.Gender.str.contains(',', regex= True, na=False)]   
    sub_m.dateAcquired = sub_m.dateAcquired.astype('str')
    years = ', '.join(sub_m.dateAcquired)
    years = sorted(list([item[:4] for item in list(set(years.split(', ')))]))[0:-1]
    for year in years:
        year_sub = sub_m[sub_m.dateAcquired == year]
        m_count = len(year_sub[year_sub.Gender.str.contains(',', regex= True, na=False)])
        df_R_nats_CL.loc[len(df_R_nats_CL.index)] = [nat, year, m_count, 'Collectives']

df_R_nats_CL.dateAcquired = df_R_nats_CL.dateAcquired.astype('int')
df_R_nats_CL = df_R_nats_CL[df_R_nats_CL.dateAcquired >0]

In [251]:
#plot bubble charts with acquisizion over decades for moma old
Rhizome_MFnat = pd.concat([df_R_nats_F,df_R_nats_M])
Rhizome_MFnat_2 = pd.concat([df_R_nats_NB, df_R_nats_CL])
Rhizome_MFnat_final = pd.concat([Rhizome_MFnat, Rhizome_MFnat_2])
Rhizome_MFnat_final['Count'] = Rhizome_MFnat_final['Count'].astype(int)

In [252]:
fig43 = px.scatter(Rhizome_MFnat_final,
                 x="dateAcquired", y="Nationality", size="Count", color="Gender" ,
                 opacity = 0.3,
                 title="Rhizome artworks acquired by Nationality and Gender",

             color_discrete_sequence=['red', 'blue'],
                 width= 900,
                 height= 700)
fig43.update_xaxes()
fig43.update_layout(
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(243, 243, 243)',
    )

fig43.show()

In [52]:
#colorscales
momacolorscale = ['#FFC0CB', '#FF69B4', '#DB7093', '#C71585', '#E6E6FA', '#DDA0DD', '#EE82EE', '#9932CC', '#8B008B', '#9370DB', '#483D8B', '#4B0082', '#FA8072', '#DC143C', '#8B0000', '#FF8C00', '#FF4500', '#FFD700', '#FFFF00', '#F0E68C', '#7FFF00', '#32CD32', '#90EE90', '#00FA9A', '#3CB371', '#006400', '#9ACD32', '#6B8E23', '#66CDAA', '#20B2AA', '#5F9EA0', '#4682B4', '#B0C4DE', '#87CEFA', '#6495ED', '#00BFFF', '#1E90FF', '#0000CD', '#191970', '#FFDEAD', '#DEB887', '#BC8F8F', '#DAA520', '#B8860B', '#D2691E', '#8B4513', '#A52A2A', '#F0FFF0', '#708090', '#2F4F4F', '#696969', '#8FBC8F', '#00CED1' ]
rhzcolorscale = ['#FFFAFA','#FFC0CB', '#FF69B4', '#DB7093', '#C71585', '#E6E6FA', '#DDA0DD', '#EE82EE', '#9932CC', '#8B008B', '#9370DB', '#483D8B', '#4B0082', '#FA8072', '#DC143C', '#8B0000', '#FF8C00', '#FF4500', '#FFD700', '#FFFF00', '#F0E68C', '#7FFF00', '#32CD32', '#90EE90', '#00FA9A', '#3CB371', '#006400', '#9ACD32', '#6B8E23', '#66CDAA', '#20B2AA', '#5F9EA0', '#4682B4', '#B0C4DE', '#87CEFA', '#6495ED', '#00BFFF', '#1E90FF', '#0000CD', '#191970', '#FFDEAD', '#DEB887', '#BC8F8F', '#DAA520', '#B8860B', '#D2691E', '#8B4513', '#A52A2A', '#F0FFF0', '#708090', '#2F4F4F', '#696969', '#8FBC8F', '#00CED1', '#F0FFF0', '#F0FFFF', '#F5F5DC', '#FFFFF0', '#FFE4E1' ]


In [127]:
# Create dimensions
gender_dim = go.parcats.Dimension(
    values=rhz_parallel.Gender,
    categoryorder='category ascending', label="Gender"
)

dept_dim = go.parcats.Dimension(values=rhz_parallel.Medium, label="Department")

continent_dim = go.parcats.Dimension(
    values=rhz_parallel.Continent, label="Global North/South", categoryarray=[0, 1],
    ticktext=['Global North', 'Global South']
)

# Create parcats trace
color = rhz_parallel.Continent;
colorscale = [[0, 'lightsteelblue'], [1, 'mediumseagreen']];

fig = go.Figure(data = [go.Parcats(dimensions=[gender_dim, dept_dim, continent_dim],
        line={'color': color, 'colorscale': colorscale},
        hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])

fig.show()

In [ ]:
rhz_parallel_female = rhz_parallel.loc[rhz_parallel['Gender'] == 'F']
rhz_parallel_female = rhz_parallel_female.rename(columns={"Medium": "Department"})
rhz_parallel_female['Source'] = 0
moma_parallel_female = moma_parallel.loc[moma_parallel['Gender'] == 'F']
moma_parallel_female['Source'] = 1
moma_rhz_parallel_female = pd.concat([rhz_parallel_female, moma_parallel_female])
moma_rhz_parallel_female

In [ ]:
rhz_parallel_female_2 = rhz_parallel.loc[rhz_parallel['Gender'] == 'F']
# Create dimensions
nat_dim = go.parcats.Dimension(
    values=rhz_parallel_female_2.Nationality,
    categoryorder='category ascending', label="Gender"
)

dept_dim = go.parcats.Dimension(values=rhz_parallel_female_2.Medium, label="Department")

fig = go.Figure(data = [go.Parcats(dimensions=[nat_dim, dept_dim],
        line={'color': color, 'colorscale': colorscale},
        hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])
fig.update_layout(height=1200)
fig.show()

In [ ]:
# Create dimensions
nat_dim = go.parcats.Dimension(
    values=moma_rhz_parallel_female.Nationality,
    categoryorder='category ascending', label="Gender"
)

dept_dim = go.parcats.Dimension(values=moma_rhz_parallel_female.Department, label="Department")

source_dim = go.parcats.Dimension(
    values=moma_rhz_parallel_female.Source, label="Source", categoryarray=[0, 1],
    ticktext=['Rhizome', 'MoMA']
)

# Create parcats trace
color = moma_rhz_parallel_female.Source;
colorscale = [[0, 'lightsteelblue'], [1, 'mediumseagreen']];

fig = go.Figure(data = [go.Parcats(dimensions=[nat_dim, dept_dim, source_dim],
        line={'color': color, 'colorscale': colorscale},
        hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])
fig.update_layout(height=1200)
fig.show()

In [ ]:
df_Moma_by_periods_top = df_Moma_by_periods.loc[df_Moma_by_periods['Count'] >= 500]
df_Moma_by_periods_bottom = df_Moma_by_periods.loc[df_Moma_by_periods['Count'] < 10]
Compare_nats_department_top = Compare_nats_department.loc[Compare_nats_department['Count'] >= 20]
Compare_nats_department_bottom = Compare_nats_department.loc[Compare_nats_department['Count'] <= 5]


In [290]:
test = Rhizome_MFnat_final.groupby(['Nationality', 'Gender'],as_index=False).agg({'Count': 'sum'})
test = test.loc[(test['Nationality'] == 'Argentine') | (test['Nationality'] == 'Brazilian') | (test['Nationality'] == 'Cameroonian') | (test['Nationality'] == 'Chilean') | (test['Nationality'] == 'Chinese') | (test['Nationality'] == 'Colombian') | (test['Nationality'] == 'Egyptian') | (test['Nationality'] == 'Indian') | (test['Nationality'] == 'Iranian') | (test['Nationality'] == 'Kenyan') | (test['Nationality'] == 'Moroccan') | (test['Nationality'] == 'South African') | (test['Nationality'] == 'Sudanese') | (test['Nationality'] == 'Uruguayan')]

In [293]:
fig11 = px.bar(test, x="Nationality", y="Count", color="Gender", facet_col_spacing=0.06,  facet_col_wrap=2, title="Nationalities by Gender in Rhizome", text='Count', barmode='overlay')
fig11.update_xaxes(matches=None, showticklabels=True)
fig11.update_yaxes(showticklabels=True)
fig11.update_traces(textposition='outside')
fig11.update_layout(height=800, width=1000)
fig11.show()




In [85]:
test = df_Moma_by_periods.groupby(['Continent','Department', 'Period'],as_index=False).agg({'Count': 'sum','Females': 'sum','Males': 'sum','Missing': 'sum','Non-Binary': 'sum'})
fig = px.bar(test, x="Continent", y="Count", color="Department", facet_col="Period", facet_col_spacing=0.06,  facet_col_wrap=2, title="TBC", barmode='group', log_y=True, text_auto=True, category_orders={"Department": ['Photography', 'Drawing & Print', 'Films', 'Media & Performance'], 'Period': ['Before 1983', 'After 1983']})
fig.update_xaxes(matches=None, showticklabels=True)
fig.update_yaxes(showticklabels=True)
fig.update_traces(textposition="outside")
fig.update_layout(height=800, width=1600)
fig.show()

**BUBBLE PLOT OF ACQUIRED VS CREATED**

In [26]:
moma_bubble = Moma_by_periods.copy()
#clean from the outlier (1700 as Date Created)
moma_all_c = moma_bubble[moma_bubble['dateCreated'] >= 1929]
moma_all_clean = moma_all_c[moma_all_c['dateAcquired'] != 0]
#filter genders
filter_out = ['M', 'F', 'NB', 'missing']
moma_collectives = moma_all_clean[moma_all_clean.Gender.isin(filter_out) == False]
collective_names = set(moma_collectives['Gender'].to_list())
moma_all_clean.loc[moma_all_clean.Gender.isin(collective_names), 'Gender'] = 'Collectives'

#group by
moma_all_counts= moma_all_clean.groupby(['Period', 'Gender','dateCreated', 'dateAcquired']).size()
moma_all_df = moma_all_counts.to_frame(name = 'size').reset_index()




In [ ]:
moma_to_divide = moma_all_df.loc[moma_all_df['Period'] == 'Before 1983']
moma_to_divide['size'] = moma_to_divide['size'].div(100).round(2)
moma_to_divide['size'] = moma_to_divide['size'].multiply(30).round(2)
moma_to_divide['size'] = moma_to_divide['size'].astype('int')
moma_full = moma_all_df.loc[moma_all_df['Period'] == 'After 1983']
moma_all_df = pd.concat([moma_full, moma_to_divide])

In [28]:
#create bubble chart w/facet
moma_before = moma_all_df[(moma_all_df['Period'] == 'Before 1983') & (moma_all_df['dateAcquired'] < 1983) & (moma_all_df['Gender'] != 'NB') & (moma_all_df['Gender'] != 'missing')]
fig = px.scatter(moma_before, x='dateCreated', y='dateAcquired', color='Gender', size='size', title= "Acquired by Creation Date for MoMA before 1983", trendline="ols")
#green line before and after
#fig.add_vline(x=1983, line_width=3, line_dash="dash", line_color="green")
#OR add inside scatter the faceting
#facet_col='Period', category_orders={"Period": ["Before 1980", "After 1980"]}
trendline="ols"
fig.update_layout(width=800, height=800)
fig.show()

#remove dpt that we dropped OR to this by dept AND facets 

#create bubble chart w/facet
moma_after = moma_all_df[(moma_all_df['Period'] == 'After 1983') & (moma_all_df['dateAcquired'] >= 1983) & (moma_all_df['Gender'] != 'missing')]
fig_2 = px.scatter(moma_after, x='dateCreated', y='dateAcquired', color='Gender', size='size', title= "Acquired by Creation Date for MoMA after 1983", trendline="ols")
#green line before and after
#fig.add_vline(x=1983, line_width=3, line_dash="dash", line_color="green")
#OR add inside scatter the faceting
#facet_col='Period', category_orders={"Period": ["Before 1980", "After 1980"]}
trendline="ols"
fig_2.update_layout(width=800, height=800)
fig_2.show()

#remove dpt that we dropped OR to this by dept AND facets 

In [408]:
#add source info
rhz_bubble = rhz_artworks.copy()
rhz_bubble['dateAcquired'] = rhz_bubble['dateAcquired'].astype('int')
rhz_bubble['dateCreated'] = rhz_bubble['dateCreated'].astype('int')
rhz_bubble['Source'] = 'Rhizome'
rhz_bubble = rhz_bubble.replace('N/B', 'NB')
moma_compare_bubble = moma_rhz_compare.copy()
moma_compare_bubble['Source'] = 'MoMA'

#combine
net_art_all= pd.concat([rhz_bubble, moma_compare_bubble])
#clean from 0
net_art_all_c =net_art_all[net_art_all['dateCreated'] >= 2000]
net_art_all_clean = net_art_all_c[net_art_all_c['dateAcquired'] != 0]

#filter genders
filter_out = ['M', 'F', 'NB', 'missing']
compare_collectives = net_art_all_clean[net_art_all_clean.Gender.isin(filter_out) == False]
compare_collective_names = set(compare_collectives['Gender'].to_list())
net_art_all_clean.loc[net_art_all_clean.Gender.isin(compare_collective_names), 'Gender'] = 'Collectives'

#group by
net_art_all_counts= net_art_all_clean.groupby(['Source', 'Gender','dateCreated', 'dateAcquired', 'Nationality']).size()
net_art_all_counts_df = net_art_all_counts.to_frame(name = 'size').reset_index()


In [ ]:
compare_to_divide = net_art_all_counts_df.loc[net_art_all_counts_df['Source'] == 'MoMA']
compare_to_divide['size'] = compare_to_divide['size'].div(10).round(2)
compare_to_divide['size'] = compare_to_divide['size'].astype(int)
rhizome = net_art_all_counts_df.loc[net_art_all_counts_df['Source'] == 'Rhizome']
net_art_all_counts_df = pd.concat([rhizome, compare_to_divide])

In [349]:
moma_rhz_parallel_both_south

,ID,dateAcquired,dateCreated,Nationality,Gender,Department,Continent,Source
73,712,2004,2004,Argentine,F,Moving Images,1,Rhizome
169,175,2005,2005,Argentine,M,Media and Performance,1,Rhizome
216,836,2001,2000,Argentine,M,Moving Images,1,Rhizome
287,347,2004,2004,Argentine,F,Media and Performance,1,Rhizome
398,648,2004,2004,South African,M,Still Images,1,Rhizome
...,...,...,...,...,...,...,...,...
138113,<NA>,2020,1996,Argentine,missing,Still Images,1,MoMA
138114,<NA>,2020,1996,Argentine,missing,Still Images,1,MoMA
138115,<NA>,2020,1996,Argentine,missing,Still Images,1,MoMA
138116,<NA>,2020,1996,Argentine,missing,Still Images,1,MoMA


In [351]:
#add source info
bubble_south = moma_rhz_parallel_both_south.copy()
bubble_south[['dateAcquired', 'dateCreated']] = bubble_south[['dateAcquired', 'dateCreated']].astype(int)
#clean from 0
bubble_south = bubble_south[bubble_south['dateCreated'] >= 2000]
bubble_south = bubble_south[bubble_south['dateAcquired'] != 0]

#group by
bubble_south = bubble_south.groupby(['Source', 'Gender','dateCreated', 'dateAcquired', 'Nationality']).size()
bubble_south = bubble_south.to_frame(name = 'size').reset_index()


In [419]:
#create bubble chart w/facet
bubble_south_rhz = bubble_south.loc[bubble_south['Source'] == 'Rhizome']
bubble_south_moma = bubble_south.loc[bubble_south['Source'] == 'MoMA']
size=bubble_south['size']
fig = px.scatter(bubble_south, y='dateCreated', x='dateAcquired', color='Nationality', size='size', symbol='Gender', facet_col='Source', title= "Acquisitions of Artworks from the Global South across Collection, by Gender and Nationality", template='seaborn')
#green line before and after
#fig.add_hline(y=2007, line_dash="dot", line_width=3, annotation_text="2007", annotation_position="bottom left", annotation_font_size=10, annotation_font_color="blue")
#fig.add_vline(x=2007, line_dash="dot", line_width=3)
#OR add inside scatter the faceting
#facet_col='Period', category_orders={"Period": ["Before 1980", "After 1980"]}
#trendline="ols"
fig.update_traces(marker=dict(
        size=size,
        sizemode='area',
        sizeref=2.*max(size)/(40.**2),
        sizemin=4
    ))
fig.update_layout(width=1400)
fig.show()
#fig.write_html(path+"file.html")
tls.get_embed('https://plotly.com/~chris/1638')

'<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plotly.com/~chris/1638.embed" height="525" width="100%"></iframe>'

In [414]:
#create bubble chart w/facet
fig = px.scatter(net_art_all_counts_df, y='dateCreated', x='dateAcquired', color='Gender', size='size', title= "Acquired by Creation Date Rhizome vs MoMA (sampled)", trendline="ols", facet_col='Source', template='seaborn')
#green line before and after
fig.add_hline(y=2007, line_dash="dot", line_width=3, 
              annotation_text="2007", 
              annotation_position="bottom left",
              annotation_font_size=10,
              annotation_font_color="blue"
             )
fig.add_vline(x=2007, line_dash="dot", line_width=3
             )
#OR add inside scatter the faceting
#facet_col='Period', category_orders={"Period": ["Before 1980", "After 1980"]}
trendline="ols"
#fig.update_layout(height=1600)

fig.show()
#fig.write_html(path+"file.html")

In [415]:
rhz_tree = pd.DataFrame(columns=['Keywords'])
rhz_kw_2 = rhizome_txt_stop_kw['Keywords'].values.tolist()
rhz_kw_2 = ' '.join(rhz_kw_2)
rhz_kw_2 = list(rhz_kw_2.split(', '))
rhz_tree['Keywords'] = rhz_kw_2
rhz_tree = rhz_tree['Keywords'].value_counts().rename_axis('Keywords').reset_index(name='Counts')
rhz_tree['Source'] = 'Rhizome'
moma_tree = pd.DataFrame(columns=['Keywords'])
moma_kw_2 = moma_full_text_stop['Keywords'].values.tolist()
moma_kw_2 = ' '.join(moma_kw_2)
moma_kw_2 = list(moma_kw_2.split(', '))
moma_tree['Keywords'] = moma_kw_2
moma_tree = moma_tree['Keywords'].value_counts().rename_axis('Keywords').reset_index(name='Counts')
moma_tree['Source'] = 'MoMA'
kw_tree = pd.concat([moma_tree, rhz_tree])
kw_tree

,Keywords,Counts,Source
0,work,614,MoMA
1,art,469,MoMA
2,artist,449,MoMA
3,made,334,MoMA
4,new,295,MoMA
...,...,...,...
13104,reinfurt,1,Rhizome
13105,tisch,1,Rhizome
13106,telecommunications,1,Rhizome
13107,participatory two,1,Rhizome


In [416]:
stopwords=['september', 'december', 'august', 'april', 'july', 'march', 'january', 'february', 'may', 'june', 'october', 'november', 'like', 'use', 'two', 'one', 'u', 'br', 'three', 'The', 'like', 'year', 'also', 'well', 'twenty', 'forms']

kw_tree_clean = kw_tree[~kw_tree['Keywords'].isin(stopwords)]

In [417]:
kw_tree_clean = kw_tree_clean.loc[kw_tree_clean['Counts'] >= 100]
fig = px.treemap(kw_tree_clean, path=['Source', 'Keywords'], values='Counts',
                  color='Source', maxdepth=2)
fig.update_traces(hovertemplate=None, textinfo='label', hoverinfo='label+value')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

